<a href="https://colab.research.google.com/github/androb/deliverables/blob/main/Interview_Playbook_as_an_OpenAI_Assistant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [359]:
# !apt-get update
# !apt-get install wkhtmltopdf pandoc libfontconfig1 libxrender1 libxext6
# !apt --fix-broken install

In [360]:
!pip install OpenAI markdown mdx_truly_sane_lists weasyprint

In [361]:
import os
from google.colab import userdata
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

# Setup

In [362]:
from openai import OpenAI

client = OpenAI()

# retrieve the Assistant
my_assistant = client.beta.assistants.retrieve("asst_3WQiw8VJ91XWikievImBqwrZ")

# create a thread
my_thread = client.beta.threads.create()


In [363]:
import time

def create_run_and_display_response(client, my_thread, my_assistant, content, file_ids=None):
    """
    Create a message, run it, and display the response.

    Args:
    client (Client): The API client.
    my_thread (Thread): The thread to post the message to.
    my_assistant (Assistant): The assistant to use for the run.
    content (str): The content of the message.
    file_ids (list, optional): List of file IDs to attach to the message.

    Returns:
    str: The response content.

    Raises:
    ValueError: If any of the parameters are invalid.
    RuntimeError: If the run does not complete successfully.
    """

    if not all([client, my_thread, my_assistant, content]):
        raise ValueError("Invalid parameters provided.")

    extra_instructions = """ ONLY return the content as markdown with no commentary what so ever."""

    # Create the message
    thread_message = client.beta.threads.messages.create(
        my_thread.id,
        role="user",
        content=content + extra_instructions,
        file_ids=file_ids or []
    )

    # Hard code the values for max_retries and wait_seconds
    max_retries = 120
    wait_seconds = 0.5

    # Create a run
    run = client.beta.threads.runs.create(
        thread_id=my_thread.id,
        assistant_id=my_assistant.id
    )

    # Check the run's status
    for _ in range(max_retries):
        run = client.beta.threads.runs.retrieve(
            thread_id=my_thread.id,
            run_id=run.id)
        if run.status == "completed":
            break
        time.sleep(wait_seconds)
    else:
        raise RuntimeError("Run did not complete successfully.")

    # Retrieve and return the response
    all_messages = client.beta.threads.messages.list(thread_id=my_thread.id)
    return all_messages.data[0].content[0].text.value + "\n\n"



# Initial File Upload and Prompt

In [364]:
# get the job description from GitHub
import time

# infra engineer example
file_path = 'jobdescription.txt'
!wget -O 'jobdescription.txt' https://raw.githubusercontent.com/androb/deliverables/main/interviewplaybook/examples/infra-engineer/job-description.md
# load it into a variable for later use
with open(file_path, 'r') as file:
    job_description = file.read()

!wget -O companyvalues.txt https://raw.githubusercontent.com/androb/deliverables/main/interviewplaybook/examples/infra-engineer/company-values.md
# time.sleep(5)


# medical assistant example
# !wget -O jobdescription.txt https://raw.githubusercontent.com/androb/deliverables/main/interviewplaybook/examples/medical-assistant/job-description.md

# Upload the file
file_company_values = client.files.create(
  file=open("companyvalues.txt", "rb"),
  purpose="assistants"
)

# Upload the file
file_job_description = client.files.create(
  file=open("jobdescription.txt", "rb"),
  purpose="assistants"
)
time.sleep(5)



--2023-12-19 00:13:50--  https://raw.githubusercontent.com/androb/deliverables/main/interviewplaybook/examples/infra-engineer/job-description.md
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4034 (3.9K) [text/plain]
Saving to: ‘jobdescription.txt’

jobdescription.txt  100%[===================>]   3.94K  --.-KB/s    in 0s      

2023-12-19 00:13:51 (33.9 MB/s) - ‘jobdescription.txt’ saved [4034/4034]

--2023-12-19 00:13:56--  https://raw.githubusercontent.com/androb/deliverables/main/interviewplaybook/examples/infra-engineer/company-values.md
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... conn

# Create the Introduction

In [365]:
# Refresh the context to ensure it has read the files
values = create_run_and_display_response(client,my_thread,my_assistant,"""
            Write a detailed description of the company values file, using the company name.
            """, [file_job_description.id, file_company_values.id])
print(values)

# Refresh the context to ensure it has read the files
competency_list = create_run_and_display_response(client,my_thread,my_assistant,"""
            Then, write the job title contained in the job description file.
            Then, create the list of technical competencies, which MUST cover the areas mentioned in the
            requirements in the job description (and no more). Create the list of behavioral
            competencies, with a focus on the company values but still covering all requirements in the
            job description. Where possible, describe a competency in a way that aligns with the company values.
            Each competency should have a title followed by a short description.
            """, [file_job_description.id, file_company_values.id])
print(competency_list)

# Y Combinator: Mission, Vision, and Values

## Mission
Y Combinator's mission is to increase the number of startups by assisting founders in their creation, helping them to build something noteworthy to raise funds on a larger scale.

## Vision
The vision of Y Combinator is a world where startups are common and are seen as being significantly productive for founders and early employees compared to working in established companies.

## Values
Y Combinator values integrity, respect, and accountability, emphasizing trust within its community. Putting founders' interests first and valuing evidence of demand for startups' solutions are also core to Y Combinator’s approach, along with a focus on launching and testing ideas quickly to respond to market changes.




In [366]:
introduction = create_run_and_display_response(client,my_thread,my_assistant,"Generate the Introduction with the Candidate Profle (only if applicable) and Interview Structure. Be sure to include mention of the competencies in the interview structure.")
print(introduction)

# Interview Playbook for Infrastructure Software Engineer

## Introduction

This interview is for the Infrastructure Software Engineer position at Y Combinator located in the San Francisco Bay Area. This role is integral in managing and evolving YC's shared infrastructure across various software products. The ideal candidate is expected to be proficient in a variety of technical competencies including MVC web application development, AWS infrastructure, Docker deployment, and Terraform, along with 5 years of industry experience. Behavioral competencies play an equally important role, reflecting Y Combinator's values of integrity, respect, accountability, founder support, and agility in launching ideas and adapting to market changes. 

## Interview Structure

The interview will consist of the following components, each designed to assess both technical and behavioral competencies aligned with Y Combinator's values:

1. Introduction (5 minutes): A welcoming approach typical of YC's respe

# Create Scoring and Evaluation Criteria

In [367]:
eval_criteria = create_run_and_display_response(client,my_thread,my_assistant,"""
    Generate the Scoring and Evaluation Critera section.
    """)
print(eval_criteria)

## Scoring and Evaluation Criteria

| Competency                  | Assessment | Comments                |
|-----------------------------|------------|-------------------------|
| MVC Web Application         |            |                         |
| AWS Infrastructure          |            |                         |
| Docker Deployment           |            |                         |
| Terraform                   |            |                         |
| Industry Experience (5+ yrs)|            |                         |
| Integrity and Respect       |            |                         |
| Accountability              |            |                         |
| Founder Support             |            |                         |
| Agility and Adaptability    |            |                         |
| Practical Assignment        |            |                         |
| Team Interaction            |            |                         |

### Rating Guide

- **Exceptional (5)**:

# Create Technical Assessment

In [368]:
technical_assessment_1 = create_run_and_display_response(client,my_thread,my_assistant,"Generate the first Technical Assessment")
print(technical_assessment_1)


## Technical Assessment

### Technical Competence 1: MVC Web Application Development
Proficiency in developing and debugging full-stack MVC (Model-View-Controller) web applications is critical for creating maintainable, efficient, and secure web services. The role requires a deep understanding of server-side and client-side interactions, database management, and the ability to architect solutions that leverage the MVC design pattern effectively.

#### Lead Question
Can you describe a time when you had to troubleshoot a challenging bug in an MVC application and how you approached solving it, including the tools and processes you used?

#### Probing Questions

- What were the most common performance bottlenecks you encountered in MVC web applications and how did you address them?
- How do you go about understanding and implementing necessary cross-cutting features like authentication and authorization in MVC apps?
- Explain how you maintain separation of concerns within MVC frameworks wh

In [369]:
technical_assessment_2 = create_run_and_display_response(client,my_thread,my_assistant,"Generate the 2nd Technical Assessment. Don't include the main section header.")
print(technical_assessment_2)

### Technical Competence 2: AWS Infrastructure
Understanding and managing cloud infrastructure within the AWS ecosystem is essential. This competency includes proficiency with services like ECS, RDS, IAM, and CloudWatch to deploy, scale, and secure web applications, ensuring reliability and optimizing for cost and performance.

#### Lead Question
Describe a past project where you were responsible for designing or optimizing the AWS infrastructure. What were the key AWS services you implemented, and what challenges did you face during the process?

#### Probing Questions

- How do you approach securing an application's environment within AWS, and can you give an example of how you've managed IAM roles and policies?
- Can you explain a scenario where you needed to scale a service using AWS and how you ensured that it was both cost-effective and resilient?
- What strategies do you use to monitor and manage the performance of web applications using AWS services, particularly CloudWatch?
- 

In [370]:
technical_assessment_3 = create_run_and_display_response(client,my_thread,my_assistant,"Generate the 3rd Technical Assessment. Don't include the main section header.")
print(technical_assessment_3)

### Technical Competence 3: Docker Deployment
The ability to containerize applications using Docker is essential for ensuring consistent, scalable, and isolated environments. Mastery in writing Dockerfiles, managing containers, and understanding Docker networking is key to improving deployment processes and maintaining application delivery consistency.

#### Lead Question
Can you share your experience with building and optimizing Dockerfiles for production environments and highlight any unique challenges you've encountered?

#### Probing Questions

- How do you manage persistent data and state across container restarts in Docker?
- Describe your strategy for optimizing Docker image sizes without compromising the performance or security of the containerized application.
- What approach do you take to maintain a smooth CI/CD pipeline with Docker, especially when dealing with multiple service dependencies?
- How do you ensure that Dockerized services successfully communicate in a microser

In [371]:
technical_assessment_4 = create_run_and_display_response(client,my_thread,my_assistant,"Generate the 4th Technical Assessment. Don't include the main section header.")
print(technical_assessment_4)

### Technical Competence 4: Terraform
Mastery in using Terraform for infrastructure as code is crucial for automating and maintaining cloud infrastructure. This involves writing, planning, and creating reproducible and modular infrastructure with Terraform scripts, and managing changes to infrastructure with minimal disruption.

#### Lead Question
Could you walk us through a complex infrastructure setup that you automated using Terraform, detailing the challenges encountered and how you overcame them?

#### Probing Questions

- How do you ensure Terraform state files are securely managed and synchronized across a distributed team?
- What is your strategy for modularizing Terraform code to be reusable for different environments or projects?
- Describe how you plan and apply incremental changes to infrastructure with Terraform in a production environment.
- Can you discuss a time when you refactored existing infrastructure with Terraform and the steps you took to ensure zero downtime?

#

In [372]:
technical_assessment_5 = create_run_and_display_response(client,my_thread,my_assistant,"Generate the 5th Technical Assessment. Don't include the main section header.")
print(technical_assessment_5)

### Technical Competence 5: Industry Experience
Having a minimum of 5 years of experience in developing and deploying web applications is crucial for understanding the full lifecycle and complexities of software development. This includes a deep knowledge of scalable architectures, security best practices, and the ability to mentor less experienced team members.

#### Lead Question
Can you discuss a complex technical project you led or contributed to significantly over the past years that showcases your depth of industry experience?

#### Probing Questions

- How has your approach to software architecture evolved over your years of experience, especially around scalability and security?
- Can you give an example of how you've mentored or influenced less experienced developers in a past role?
- Reflect on a significant technological shift you've navigated in your career. How did you adapt and what did you learn?
- What’s the most impactful technical decision you've made in your career a

In [373]:
technical_assessment = technical_assessment_1 + technical_assessment_2 + technical_assessment_3 + technical_assessment_4 + technical_assessment_5

# Create Behavioral Assessment

In [374]:
behavioral_assessment_1 = create_run_and_display_response(client,my_thread,my_assistant,"Generate the 1st Behavioral Assessment section.")
print(behavioral_assessment_1)

## Behavioral Assessment

### Behavioral Competence 1: Integrity and Respect
Y Combinator values integrity as a cornerstone of their organization, ensuring that all members act ethically and with respect towards each other. Candidates must display a history of ethical behavior and show respect in their interactions, decisions, and work.

#### Lead Question
Tell me about a time when you were faced with an ethical dilemma in your professional life. How did you handle it, and what was the outcome?

#### Probing Questions

- How do you ensure that your personal values are in alignment with your professional responsibilities and conduct?
- Can you provide an example of how you have handled a sensitive or confidential matter at work?
- Describe a moment when you had to give difficult feedback to a colleague. How did you approach the situation to maintain respect and integrity?
- What strategies do you use to foster an environment of trust and respect within a team?

#### Scenario
Imagine you

In [375]:
behavioral_assessment_2 = create_run_and_display_response(client,my_thread,my_assistant,"Generate the 2nd Behavioral Assessment section. Don't include the main section header.")
print(behavioral_assessment_2)

### Behavioral Competence 2: Accountability
Accountability is a key value at Y Combinator, with an emphasis on taking ownership of one's actions and responsibilities within the organization. This competency is about being reliable, meeting deadlines, and following through on commitments made to peers and the company.

#### Lead Question
Share an example of how you've taken ownership of a problem that was not originally within your scope of responsibilities. What was the issue and what steps did you take to resolve it?

#### Probing Questions

- How do you prioritize your tasks and manage expectations when you have multiple high-priority commitments?
- Tell me about a time when you made a mistake at work. How did you handle it and what was the outcome?
- Describe how you keep yourself accountable in a remote or unsupervised work environment.
- Can you provide an instance when you proactively updated stakeholders on a project's progress or potential setbacks?

#### Scenario
Imagine you'r

In [376]:
behavioral_assessment_3 = create_run_and_display_response(client,my_thread,my_assistant,"Generate the 3rd Behavioral Assessment section. Don't include the main section header.")
print(behavioral_assessment_3)

### Behavioral Competence 3: Founder Support
Y Combinator places high importance on the ability to support and prioritize the interests of founders, reflecting a commitment to the startup ecosystem. This competency involves demonstrating a service-oriented mindset and taking actions that align with the founders' vision and goals.

#### Lead Question
Tell us about a situation where you had to support an entrepreneur or a founding team. What was required from you, and how did you contribute to their success?

#### Probing Questions

- How have you handled a scenario where a founder's vision conflicted with your own professional opinion or experience?
- Describe a time when you went above and beyond to ensure that a project or task aligned with the overarching goals of a founder.
- Can you provide an example of how you helped a startup or a founding team navigate a challenging period?
- In what ways have you contributed to creating a supportive environment that fosters innovation and risk

In [377]:
behavioral_assessment_4 = create_run_and_display_response(client,my_thread,my_assistant,"Generate the 4th Behavioral Assessment section. Don't include the main section header.")
print(behavioral_assessment_4)

### Behavioral Competence 4: Agility and Adaptability
In an ever-changing market, Y Combinator values the ability to quickly adapt and pivot strategies. This competency focuses on the candidate's agility to respond to new information and the adaptability to embrace change effectively.

#### Lead Question
Can you describe a time when you had to quickly change your approach due to new or unexpected developments on a project?

#### Probing Questions

- How do you stay informed of the latest trends and updates relevant to your field, and how do you apply this knowledge?
- What is your process for managing uncertainty and making decisions in fast-paced or ambiguous situations?
- Can you provide an example of an initiative you took that required you to learn a new skill or adapt to a new environment?
- Explain a scenario where you had to persuade your team to adopt a significant change to a project plan or strategy.

#### Scenario
Y Combinator decides to pivot the technology stack for a key 

In [378]:
behavioral_assessment_5 = create_run_and_display_response(client,my_thread,my_assistant,"Generate the 5th Behavioral Assessment section. Don't include the main section header.")
print(behavioral_assessment_5)

### Behavioral Competence 5: Evidence of Demand and Market Responsiveness
Y Combinator believes in the significance of rapid testing and responsiveness to market demands. Candidates should demonstrate the ability to validate ideas swiftly, pivot based on user feedback, and handle shifts in market trends.

#### Lead Question
Describe a time when you had to pivot or make a significant change to a product based on market feedback or a sudden shift in user needs.

#### Probing Questions

- How do you validate new ideas or features with real users, and can you share a successful example of this process?
- What methodologies do you use to gather and process user feedback and market data?
- Have you ever had to deprioritize or eliminate a feature or project due to changing market demands? How did you approach this decision?
- How do you balance the need for swift responsiveness to the market with the strategic vision of the product?

#### Scenario
A product you're developing at Y Combinator h

In [379]:
behavioral_assessment = behavioral_assessment_1 + behavioral_assessment_2 + behavioral_assessment_3 + behavioral_assessment_4 + behavioral_assessment_5

# Create Practical Assignment

In [380]:
practical_assignment = create_run_and_display_response(client,my_thread,my_assistant,"Generate the Practical Assignment section.")
print(practical_assignment)


## Practical Assignment

The practical assignment involves the candidate migrating a Rails application from Capistrano and EC2 to Docker and ECS. The objective is to assess the candidate's practical skills in containerization and cloud service deployment, as well as their familiarity with AWS, Docker, and orchestration using ECS.

Evaluation will be based on the candidate's approach to creating Dockerfiles, setting up ECS services, handling persistence and state management, and ensuring minimal downtime during the migration process. 

To assess the candidate's approach and solution, the interviewer should consider the following:

- How well the candidate understands the nuances of containerizing a Rails application.
- The ability to maintain a clean and efficient development workflow leveraging Docker and ECS.
- Strategies for migrating live databases and user data with zero data loss.
- Practices implemented to monitor and verify the successful operation of the application post-migrat

# Create Team Interaction

In [381]:
team_interaction = create_run_and_display_response(client,my_thread,my_assistant,"Generate the Team Interaction section.")
print(team_interaction)

## Team Interaction

Team interaction provides an opportunity for the candidate to engage with potential future colleagues. It serves to assess how well they would integrate into the team, their collaborative skills, and their fit within Y Combinator's company culture.

This interaction should be a mix of informal discussion and small problem-solving exercises relevant to the team's current projects. Proposed topics can include but are not limited to: brainstorming sessions for new features, commenting on the existing codebase's structure, or discussing approaches to ongoing challenges faced by the current team.

During the interaction, observe the candidate's:

- Communication style and ability to articulate thoughts.
- Ability to listen and build upon others' ideas.
- Approach to giving and receiving constructive feedback.
- Comprehension of and reaction to the team's current projects and methodologies.

The aim is to gauge mutual compatibility and to envision how the candidate would

# Create Candidate Q&A

In [382]:
candidate_questions = create_run_and_display_response(client,my_thread,my_assistant,"""
    Generate the Candidate Q&A section. Include example answers.
    """)
print(candidate_questions)

## Candidate Q&A

Candidates often have queries about the role, company, and expectations. The interviewer should be prepared to provide comprehensive responses.

- **Candidate Question**: Can you tell me more about the day-to-day responsibilities of this role?
  - **Answer**: As an Infrastructure Software Engineer at Y Combinator, you'll be actively involved in developing, deploying, and managing our shared infrastructure, with a focus on efficiency, scalability, and security using AWS, Docker, and Terraform among other technologies.

- **Candidate Question**: What are the growth opportunities within Y Combinator for someone in this position?
  - **Answer**: Y Combinator supports professional growth with opportunities to lead projects, mentor peers, and potentially move into larger leadership roles within the technical team. We encourage continuous learning and provide resources for expanding your skill set.

- **Candidate Question**: How does the team at Y Combinator stay up-to-date 

# Finalize the Playbook HTML

## Create a markdown version

For use in GitHub as an example

In [390]:
playbook_md = (introduction + "\n\n" +
            job_description + "\n\n" +
            eval_criteria + "\n\n" +
            technical_assessment + "\n\n" +
            behavioral_assessment + "\n\n" +
            practical_assignment + "\n\n" +
            candidate_questions + "\n\n" +
            team_interaction + "\n\n" )

# Write playbook to a text file
with open('playbook.md', 'w') as file:
    file.write(playbook_md)

## Convert the markdown to HTML

We are doing this in chunks so that we can do more useful things with it in the HTML output

In [391]:
import markdown

# Convert Markdown to HTML
html_playbook = markdown.markdown(playbook_md, extensions=['extra','mdx_truly_sane_lists'])

# Convert Markdown to HTML
html_job_description = markdown.markdown(job_description, extensions=['extra','mdx_truly_sane_lists'])
html_introduction = markdown.markdown(introduction, extensions=['extra','mdx_truly_sane_lists'])
html_technical_assessment = markdown.markdown(technical_assessment, extensions=['extra','mdx_truly_sane_lists'])
html_behavioral_assessment = markdown.markdown(behavioral_assessment, extensions=['extra','mdx_truly_sane_lists'])
html_practical_assignment = markdown.markdown(practical_assignment, extensions=['extra','mdx_truly_sane_lists'])
html_candidate_questions = markdown.markdown(candidate_questions, extensions=['extra','mdx_truly_sane_lists'])
html_team_interaction = markdown.markdown(team_interaction, extensions=['extra','mdx_truly_sane_lists'])
html_eval_criteria = markdown.markdown(eval_criteria, extensions=['extra','mdx_truly_sane_lists'])


html_technical_assessment_1 = markdown.markdown(technical_assessment_1, extensions=['extra','mdx_truly_sane_lists'])
html_technical_assessment_2 = markdown.markdown(technical_assessment_2, extensions=['extra','mdx_truly_sane_lists'])
html_technical_assessment_3 = markdown.markdown(technical_assessment_3, extensions=['extra','mdx_truly_sane_lists'])
html_technical_assessment_4 = markdown.markdown(technical_assessment_4, extensions=['extra','mdx_truly_sane_lists'])
html_technical_assessment_5 = markdown.markdown(technical_assessment_5, extensions=['extra','mdx_truly_sane_lists'])

html_behavioral_assessment_1 = markdown.markdown(behavioral_assessment_1, extensions=['extra','mdx_truly_sane_lists'])
html_behavioral_assessment_2 = markdown.markdown(behavioral_assessment_2, extensions=['extra','mdx_truly_sane_lists'])
html_behavioral_assessment_3 = markdown.markdown(behavioral_assessment_3, extensions=['extra','mdx_truly_sane_lists'])
html_behavioral_assessment_4 = markdown.markdown(behavioral_assessment_4, extensions=['extra','mdx_truly_sane_lists'])
html_behavioral_assessment_5 = markdown.markdown(behavioral_assessment_5, extensions=['extra','mdx_truly_sane_lists'])



 ## Create the style for the document

In [405]:
styles= """

   body {
        font-family: Source Sans Pro;
        font-size: 10pt;
       /* column-count: 2; */
       /* column-gap: 20px; */
    }

    @page {
        margin: 0.5in;
        size: Letter;
        @bottom-right {
          content: counter(page);
          font-size: 10pt;
          color: #333333;
        }
    }

    h1, h2, h3, h4 {
        font-family: Libre Baskerville;
        margin-top: 1em;
        margin-bottom: 0.5em;
    }
    h1 {
        font-size: 24pt;
        margin-top: 3em;
        page-break-after: avoid;
    }
    .intro h1{
        font-size: 50pt;
        margin-top: 3em;
        page-break-after: always;
    }
    @media print {
      /* Define styles for the cover page */
      .title {
        position: fixed;
        left: 0;
        top: 0;
        width: 100%;
        height: 100%;
        background-color: #fff; /* Change as needed */
        color: #000; /* Change as needed */
        text-align: center;
        padding: 50px;
      }
    }
    .publication-date {
      position: absolute;
      bottom: 10px; /* adjust as needed */
      width: 100%;
      text-align: center;
      font-size: 12pt; /* adjust as needed */
      color: #333; /* adjust as needed */
    }


    h2 {
        font-size: 18pt;
        page-break-after: avoid;
    }
    h3 {
        font-size: 14pt;
        page-break-after: avoid;
    }
    h4 {
        font-size: 12pt;
        page-break-after: avoid;
    }
    p {
        margin-bottom: 1em;
    }


    h1, h2, h3, h4 {
        page-break-after: avoid;
    }

    p, table, figure {
        page-break-inside: avoid;
    }
    table {
        width: 100%;
        border-collapse: collapse;
        margin-bottom: 1em;
        page-break-before: avoid;
    }
    th, td {
        border: 1px solid #ddd;
        padding: 8px;
        text-align: left;
        font-size: 10pt;
    }
    th {
        background-color: #f2f2f2;
        font-weight: bold;
    }
    tr:nth-child(even) {{background-color: #f9f9f9;}}

    ul, ol {
        margin-bottom: 1em;
        margin-left: 0;
        padding-left:1.5em;
    }
    li {
        margin-bottom: 0.5em;
        margin-left: 0em;
    }

    .main {
      page-break-before: always;
    }

    .dotted-line {
        border-bottom: 1px dotted black;
        width: 100%;
        margin-top: 20px; /* This ensures spacing between each line */
        margin-bottom: 20px; /* Adjust if needed */
    }

    @media print {
        .dotted-line {
            margin-top: 20px; /* Adjust as needed for printing */
            margin-bottom: 20px; /* Adjust as needed for printing */
        }
    }

    .rating-container {
        display: flex;
        align-items: flex-end;
        margin-top: 20px;
        margin-bottom: 20px;
    }

    .rating-container label {
        margin-right: 10px;
        font-size: 10pt;
        font-family: Source Sans Pro;
    }

    .short-dotted-line {
        border-bottom: 1px dotted black;
        width: 150px; /* Adjust the width as needed */
        margin-left: 5px;
    }}

    @media print {
        .short-dotted-line {
            width: 150px; /* Adjust for printing if needed */
        }
    }
"""

## Create some useful HTML snippets

In [411]:

html_notes = """<h4>Notes</h4><div class='notes'>
    <div class="dotted-line"></div>
    <div class="dotted-line"></div>
    <div class="dotted-line"></div>
    <div class="dotted-line"></div>
    <div class="dotted-line"></div>
    <div class="dotted-line"></div>
    <div class="dotted-line"></div>
    <div class="dotted-line"></div>
    <div class="rating-container">
    <label for="rating-line">Rating (1-5):</label>
    <div id="rating-line" class="short-dotted-line"></div>
    </div>
</div>"""

html_short_notes = """<h4>Notes</h4><div class='notes'>
    <div class="dotted-line"></div>
    <div class="dotted-line"></div>
    <div class="dotted-line"></div>
    <div class="dotted-line"></div>
    <div class="rating-container">
    <label for="rating-line">Rating (1-5):</label>
    <div id="rating-line" class="short-dotted-line"></div>
    </div>
</div>"""

from datetime import datetime

# Getting today's date in a nicely formatted string for US English
publication_date = datetime.now().strftime("%B %d, %Y")

## Build the HTML

In [412]:
# Define the HTML template
html_content = rf"""
<meta charset="utf-8">
<html>
<head>
    <link href="https://fonts.googleapis.com/css2?family=Source%20Sans%20Pro:wght@400;700&display=swap" rel="stylesheet">
    <link href="https://fonts.googleapis.com/css2?family=Libre%20Baskerville:wght@400;700&display=swap" rel="stylesheet">
    <style>
    {styles}
    </style>
</head>
<body>
    <section class='intro'>
    <div class="publication-date">Published on: {publication_date}</div>
    {html_introduction}
    </section>
    <section class='main'>
    {html_job_description}
    </section>
    <section class='main'>
    {html_eval_criteria}
    </section>
    <section class='main'>
      {html_technical_assessment_1}
     {html_notes}
    </section>
    <section class='main'>
      {html_technical_assessment_2}
     {html_notes}
    </section>
        <section class='main'>
      {html_technical_assessment_3}
     {html_notes}
    </section>
        <section class='main'>
      {html_technical_assessment_4}
     {html_notes}
    </section>
        <section class='main'>
      {html_technical_assessment_5}
     {html_notes}
    </section>
    <section class='main'>
    {html_behavioral_assessment_1}
     {html_notes}
    </section>
    <section class='main'>
    {html_behavioral_assessment_2}
     {html_notes}
    </section>
        <section class='main'>
    {html_behavioral_assessment_3}
     {html_notes}
    </section>
        <section class='main'>
    {html_behavioral_assessment_4}
     {html_notes}
    </section>
        <section class='main'>
    {html_behavioral_assessment_5}
     {html_notes}
    </section>
    <section class='main'>
    {html_practical_assignment}
    {html_notes}
    </section>
    <section class='main'>
    {html_team_interaction}
    {html_notes}
    </section>
    <section class='main'>
    {html_candidate_questions}
    </section>

</body>
</html>"""

# Write HTML string to a file
with open('playbook.html', 'w') as file:
    file.write(html_content)



Display the HTML content here so we can review it

In [413]:
# Display the Markdown-formatted text in the notebook
from IPython.display import display, HTML
display(HTML(html_content))

Competency,Assessment,Comments
MVC Web Application,,
AWS Infrastructure,,
Docker Deployment,,
Terraform,,
Industry Experience (5+ yrs),,
Integrity and Respect,,
Accountability,,
Founder Support,,
Agility and Adaptability,,
Practical Assignment,,


# Create the Documents

Create the Word document

In [403]:
# Write HTML string to a file
with open('playbook.html', 'w') as file:
    file.write(html_content)

# Convert HTML to Docx
!pandoc -s playbook.html -o playbook.docx

# from google.colab import files
# files.download('playbook.docx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Create the PDF

In [414]:
from weasyprint import HTML

# Convert HTML to PDF using WeasyPrint
#HTML('playbook.html').write_pdf('playbook.pdf', stylesheets=['style.css'], presentational_hints=True)
HTML('playbook.html').write_pdf('playbook.pdf')

from google.colab import files
files.download('playbook.pdf')

# Convert HTML to alternative PDF
#!pandoc -s playbook.html --pdf-engine=wkhtmltopdf -o playbook.pdf

# Convert HTML to alternative PDF
#!wkhtmltopdf --page-size Letter --footer-html footer.html --margin-top 15mm --margin-bottom 15mm --margin-left 15mm --margin-right 15mm playbook.html playbook.pdf

# Convert HTML to PDF
##import pdfkit
# pdfkit.from_string(html_content, 'playbook.pdf')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>